In [354]:
import numpy as np
import pandas as pd
import ast                  #Used in tag creation

In [39]:
movies = pd.read_csv('tmdb_5000_movies_raw.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')

In [40]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [41]:
movies.shape

(4803, 20)

In [42]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [43]:
credits.shape

(4803, 4)

In [44]:
movies = movies.merge(credits, on='title')

In [45]:
movies.shape

(4809, 23)

In [46]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

In [47]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [48]:
#These are the parameters we will be keeping in order to make the decision#
# budget                4809 non-null   int64  
# genres                4809 non-null   object 
# id                    4809 non-null   int64  
# keywords              4809 non-null   object 
# original_language     4809 non-null   object  
# overview              4806 non-null   object 
# popularity            4809 non-null   float64
# production_companies  4809 non-null   object  
# release_date          4808 non-null   object  
# runtime               4807 non-null   float64 
# title                 4809 non-null   object 
# cast                  4809 non-null   object 
# crew               4809 non-null   object 

In [49]:
movies= movies[['genres','id','keywords','original_language','overview','popularity','production_companies','release_date','runtime','title','cast','crew']]

In [50]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                4809 non-null   object 
 1   id                    4809 non-null   int64  
 2   keywords              4809 non-null   object 
 3   original_language     4809 non-null   object 
 4   overview              4806 non-null   object 
 5   popularity            4809 non-null   float64
 6   production_companies  4809 non-null   object 
 7   release_date          4808 non-null   object 
 8   runtime               4807 non-null   float64
 9   title                 4809 non-null   object 
 10  cast                  4809 non-null   object 
 11  crew                  4809 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 488.4+ KB


In [51]:
#Creating tags out of the shortlisted parameters

In [52]:
movies.dropna(inplace=True)

In [53]:
movies.shape

(4805, 12)

In [54]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [55]:
#for extracting tags out of array of dictionary we can create a function
def extractTag(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [56]:
movies['genres']=movies['genres'].apply(extractTag)
movies['keywords']=movies['keywords'].apply(extractTag)
movies['production_companies']=movies['production_companies'].apply(extractTag)

In [57]:
#function to extract names of the top 3 leading casts of the movie
def extractCast(obj):
    L=[]
    count = 0
    for i in ast.literal_eval(obj):
        if count !=3:
            L.append(i['name'])
            count +=1
        else:
            break
    return L

In [58]:
movies['cast']=movies['cast'].apply(extractCast)

In [59]:
movies.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [60]:
#for extracting tags of crew members(taking director key only)
def extractCrew(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [61]:
movies['crew']= movies['crew'].apply(extractCrew)

In [62]:
movies.iloc[0].overview

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [63]:
movies['overview']= movies['overview'].apply(lambda x: x.split())

In [64]:
#Now we have to look for one problem that there can be two tags having more than one words
#And if two tags have the same sub tags then the recommendation may suffer problem

#Solution: Remove spaces and create a single word tag for every tag obtained till now

In [65]:
movies['genres']= movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords']= movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['overview']= movies['overview'].apply(lambda x: [i.replace(" ","") for i in x])
movies['production_companies']= movies['production_companies'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast']= movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']= movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [66]:
movies.shape

(4805, 12)

In [67]:
movies['tags']=movies['genres']+ movies['keywords']+movies['overview']+movies['production_companies']+movies['cast']+movies['crew']

In [68]:
movies.head()

,genres,id,keywords,original_language,overview,popularity,production_companies,release_date,runtime,title,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",en,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[IngeniousFilmPartners, TwentiethCenturyFoxFil...",2009-12-10,162.0,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",en,"[Captain, Barbossa,, long, believed, to, be, d...",139.082615,"[WaltDisneyPictures, JerryBruckheimerFilms, Se...",2007-05-19,169.0,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",en,"[A, cryptic, message, from, Bond’s, past, send...",107.376788,"[ColumbiaPictures, Danjaq, B24]",2015-10-26,148.0,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dccomics, crimefighter, terrorist, secretiden...",en,"[Following, the, death, of, District, Attorney...",112.312950,"[LegendaryPictures, WarnerBros., DCEntertainme...",2012-07-16,165.0,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,"[Action, Adventure, ScienceFiction]",49529,"[basedonnovel, mars, medallion, spacetravel, p...",en,"[John, Carter, is, a, war-weary,, former, mili...",43.926995,[WaltDisneyPictures],2012-03-07,132.0,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction, basedonnov..."


In [69]:
newMovies= movies[['id','title','tags']]

In [70]:
newMovies.head()

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."


In [72]:
newMovies.to_csv(r'C:\Users\kames\Desktop\NewMovies.csv', index=False)

In [379]:
newMovies= pd.read_csv('NewMovies.csv')

In [380]:
newMovies.head()

,id,title,tags
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'ScienceFic..."
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action', 'ocean', 'd..."
2,206647,Spectre,"['Action', 'Adventure', 'Crime', 'spy', 'based..."
3,49026,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller', 'dcco..."
4,49529,John Carter,"['Action', 'Adventure', 'ScienceFiction', 'bas..."


In [382]:
#Creating a string out of tags
newMovies['tags']= newMovies['tags'].apply(lambda x: " ".join(x))
newMovies['tags']= newMovies['tags'].apply(lambda x: x.replace(' ',''))

In [383]:
newMovies['tags']= newMovies['tags'].apply(lambda x: x.lower())

In [384]:
newMovies.iloc[0].tags

"['action','adventure','fantasy','sciencefiction','cultureclash','future','spacewar','spacecolony','society','spacetravel','futuristic','romance','space','alien','tribe','alienplanet','cgi','marine','soldier','battle','loveaffair','antiwar','powerrelations','mindandsoul','3d','in','the','22nd','century,','a','paraplegic','marine','is','dispatched','to','the','moon','pandora','on','a','unique','mission,','but','becomes','torn','between','following','orders','and','protecting','an','alien','civilization.','ingeniousfilmpartners','twentiethcenturyfoxfilmcorporation','duneentertainment','lightstormentertainment','samworthington','zoesaldana','sigourneyweaver','jamescameron']"

In [360]:
#'action','actions' are similar but will act as different tag
# We will nltk library to solve this issue
!pip install nltk

In [385]:
newMovies.shape

(4805, 3)

In [386]:
import nltk

In [387]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [388]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [389]:
newMovies['tags']= newMovies['tags'].apply(stem)

In [390]:
#Now we need to vectorise our tags by the method of Text Vectorization- Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=3000, stop_words='english')

In [391]:
vector= cv.fit_transform(newMovies['tags']).toarray()

In [392]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [393]:
vector.shape

(4805, 3000)

In [394]:
cv.get_feature_names()

['000',
 '10',
 '11',
 '12',
 '13',
 '14',
 '1492pictures',
 '15',
 '16',
 '17',
 '18',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '19th',
 '19thcentury',
 '20',
 '20th',
 '24',
 '30',
 '3d',
 '40',
 '40acres',
 '50',
 'aaroneckhart',
 'abandoned',
 'ability',
 'able',
 'abuse',
 'academy',
 'accepts',
 'accident',
 'accidentally',
 'account',
 'accused',
 'ace',
 'act',
 'action',
 'actions',
 'activities',
 'actor',
 'actors',
 'actress',
 'actually',
 'adam',
 'adamsandler',
 'adaptation',
 'addiction',
 'adopted',
 'adoption',
 'adult',
 'adultery',
 'adventure',
 'adventures',
 'advertising',
 'advice',
 'affair',
 'afghanistan',
 'africa',
 'african',
 'aftercreditsstinger',
 'afterlife',
 'age',
 'aged',
 'agency',
 'agent',
 'agents',
 'aging',
 'ago',
 'agree',
 'agrees',
 'ahead',
 'aid',
 'aided',
 'air',
 'airplane',
 'airport',
 'al',
 'alaska',
 'albert',
 'alcohol',
 'alcoholic',
 'alcoholism',
 'alconentertainment',
 'alecbaldwin',
 'alex',
 'alice',
 

In [395]:
from sklearn.metrics.pairwise import cosine_similarity

In [396]:
similarity = cosine_similarity(vector)

In [397]:
similarity[0]

array([1.        , 0.09583148, 0.06277648, ..., 0.0270666 , 0.02988072,
       0.        ])

In [398]:
sorted(list(enumerate(similarity[0])), reverse=True, key= lambda x: x[1])[1:6]

[(1216, 0.346274924279844),
 (507, 0.2958682036329227),
 (539, 0.29424494316824984),
 (3730, 0.28128433856309726),
 (61, 0.2788866755113585)]

In [399]:
def recommend(movie):
    movie_index= newMovies[newMovies['title']== movie].index[0]
    distances= similarity[movie_index]
    movie_list= sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(newMovies.iloc[i[0]].title)

In [400]:
recommend('Avatar')

Aliens vs Predator: Requiem
Independence Day
Titan A.E.
Falcon Rising
Jupiter Ascending


In [401]:
similarity.shape

(4805, 4805)